In [1]:
import pandas as pd
from IPython.core.display import HTML
from IPython.core.display import Image 
%load_ext music21.ipython21.ipExtension
import numpy as np
import music21 as mu
import pymongo
from fractions import Fraction
%pylab inline
from pymongo import MongoClient
from bs4 import BeautifulSoup
import re
import pydash as py_
pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (15, 5)
import math
import music21 as mu
from functools import partial
import json
# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)


Populating the interactive namespace from numpy and matplotlib


In [2]:
muAllTheThingsYouAre = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAllTheThingsYouAre.xml')
muAutumnLeaves = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAutumnLeaves.xml')
muDaysOfWineAndRoses = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettDaysOfWineAndRoses.xml')
muGroovingHigh = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettGroovinHigh.xml')
muIfIWereABell = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettIfIWereABell.xml')
muSomedayMyPrinceWillCome = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettSomedayMyPrinceWillCome.xml')
muStellaByStarlight = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettStellaByStarlight.xml')
muArmandosRhumba = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/ChickCoreaArmandosRhumba.xml')

In [3]:

def getDurationsForMongoQueries(data_row):
    valueToCheck = data_row['duration']
    try:
        valueToCheck = int(valueToCheck)
    except:
        valeToCheck = -1
    if valueToCheck == 21:
        return Fraction(1,48) #Thirty Second note triplet
    
    elif valueToCheck == 32:
        return Fraction(1,32)  #Thirty second note
    
    elif valueToCheck == 42:
        return Fraction(1,24)  #Sixteenth note triplet
    
    elif valueToCheck == 43:
        return Fraction(1,24)  #Sixteenth note triplet

    elif valueToCheck == 51:
        return Fraction(1,20)  #sixteenth note quintuplet
    
    elif valueToCheck == 52:
        return Fraction(1,20)  #sixteenth note quintuplet    
    
    elif valueToCheck == 56:
        return Fraction(1,18)  #eighth note triplet of quarter note triplet
    
    elif valueToCheck == 57:
        return Fraction(1,18)  #eighth note triplet of quarter note triplet
    
    elif valueToCheck == 64:
        return Fraction(1,16)       #sixteenth note
    
    elif valueToCheck == 73:
        return Fraction(1,14)       #Eighth note septuplet
    
    elif valueToCheck == 74:
        return Fraction(1,14)       #Eighth note septuplet
    
    elif valueToCheck == 85:
        return Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 86:
        return Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 96:   #dotted sixteenth note
        return Fraction(3,32)
    
    elif valueToCheck == 102:     #eighth note quintuplet
        return Fraction(1,10)
    
    elif valueToCheck == 103:     #eighth note quintuplet
        return Fraction(1,10)
    
    elif valueToCheck == 128:   # eighth note
        return Fraction(1,8)
    
    elif valueToCheck == 170:   #quarter note triplet
        return Fraction(1,6)
    
    elif valueToCheck == 171:   #quarter note triplet
        return Fraction(1,6)
    
    elif valueToCheck == 192:   #dotted eighth note
        return Fraction(3,16)    
    
    elif valueToCheck == 213:   #dotted eighth note
        return Fraction(5,24)    
    
    elif valueToCheck == 214:   #dotted eighth note
        return Fraction(5,24)    
    
    elif valueToCheck == 256:   #quarter note
        return Fraction(1,4)
    
    elif valueToCheck == 320:   #dotted eighth note
        return Fraction(5,16)   
    
    elif valueToCheck == 342:   #half note triplet
        return Fraction(1,3)
    
    elif valueToCheck == 384:   #dotted quarter note
        return Fraction(3,8)
    
    elif valueToCheck == 406:   #dotted quarter note
        return Fraction(19,48)
    
    elif valueToCheck == 470:   #dotted quarter note
        return Fraction(11,24)
    
    elif valueToCheck == 512:   #half note
        return Fraction(1,2)
    
    elif valueToCheck == 576:   #half note
        return Fraction(9,16)
    
    elif valueToCheck == 597:
        return Fraction (7,12)
    
    elif valueToCheck == 598:   #half note
        return Fraction(7,12)
    
    elif valueToCheck == 640:   #half note
        return Fraction(5,8)
    
    elif valueToCheck == 682:   #half note
        return Fraction(2,3)
    
    elif valueToCheck == 768:   #dotted half note
        return Fraction(3,4)
    
    elif valueToCheck == 896:   #dotted half note
        return Fraction(7,8)
    
    elif valueToCheck == 1024:   #whole note
        return Fraction(1,1)
    
    elif valueToCheck == 1365:   #whole note
        return Fraction(4,3)
    
    elif valueToCheck == 1622:   #whole note
        return Fraction(19,12)
    
    elif valueToCheck == 1920:   #whole note
        return Fraction(15,8)
    
    else:
        print 'Fix needed', data_row
        return Fraction(1)


    
def getDurationNamesForMongoQueries(data_row):
    valueToCheck = data_row['duration']
    try:
        valueToCheck = int(valueToCheck)
    except:
        valeToCheck = -1
    if valueToCheck == 21:
        return 'Thirtysecond note triplet' # Fraction(21,1024) #Thirty Second note triplet
    
    elif valueToCheck == 32:
        return 'Thirtysecond note' #Fraction(1,32)  #Thirty second note
    
    elif valueToCheck == 42:
        return 'Sixteenth note triplet' #Fraction(21,512)  #Sixteenth note triplet

    elif valueToCheck == 43:
        return 'Sixteenth note triplet' #Fraction(21,512)  #Sixteenth note triplet
    
    elif valueToCheck == 51:
        return 'Sixteenth note quintuplet' #Fraction(51,1024)  #sixteenth note quintuplet
    
    elif valueToCheck == 52:
        return 'Sixteenth note quintuplet' #Fraction(51,1024)  #sixteenth note quintuplet
    
    
    elif valueToCheck == 56:
        return 'Eighth note triplet of quarter note triplet' #Fraction(51,1024)
    elif valueToCheck == 57:
        return 'Eighth note triplet of quarter note triplet' #Fraction(51,1024)
    
    elif valueToCheck == 64:
        return 'Sixteenth note' #Fraction(1,16)       #sixteenth note
    
    elif valueToCheck == 73:
        return 'Eighth note septuplet' #Fraction(73,1024)       #sixteenth note
    
    elif valueToCheck == 74:
        return 'Eighth note septuplet' #Fraction(73,1024)       #sixteenth note
    
    elif valueToCheck == 85:
        return 'Eighth note triplet' # Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 86:
        return 'Eighth note triplet' # Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 96:   #dotted sixteenth note
        return 'Dotted sixteenth note' #Fraction(3,32)
    
    elif valueToCheck == 102:     #eighth note quintuplet
        return 'Eighth note quintuplet' #Fraction(51,512)

    elif valueToCheck == 103:     #eighth note quintuplet
        return 'Eighth note quintuplet' #Fraction(51,512)
    
    elif valueToCheck == 128:   # eighth note
        return 'Eighth note' #Fraction(1,8)
    
    elif valueToCheck == 170:   #quarter note triplet
        return 'Quarter note triplet' #Fraction(1,6)
    
    elif valueToCheck == 171:   #quarter note triplet
        return 'Quarter note triplet' #Fraction(1,6)
    
    elif valueToCheck == 192:   #dotted eighth note
        return 'Dotted eighth note' #Fraction(3,16)  
    
    elif valueToCheck == 213:   #dotted eighth note
        return 'Quarter note triplet tied to a sixteenth note triplet'    
    
    elif valueToCheck == 214:   #dotted eighth note
        return 'Quarter note triplet tied to a sixteenth note triplet'  
    
    elif valueToCheck == 256:   #quarter note
        return 'Quarter note' #Fraction(1,4)
    
    elif valueToCheck == 320:   #dotted eighth note
        return 'Quarter note tied to a sixteenth note'     

    elif valueToCheck == 342:   #dotted quarter note
        return 'Half note triplet'    
    
    elif valueToCheck == 384:   #dotted quarter note
        return 'Dotted quarter note' #Fraction(3,8)
    
    elif valueToCheck == 406:   #dotted quarter note
        return 'Half note triplet tied to a sixteenth note' #Fraction(3,8)
    
    elif valueToCheck == 470:   #dotted quarter note
        return 'Half note triplet tied to an eighth note' #Fraction(3,8)
    
    elif valueToCheck == 512:   #half note
        return 'Half note' #Fraction(1,2)
    
    elif valueToCheck == 576:   #half note
        return 'Half note tied to a sixteenth note' #Fraction(1,2)
    
    elif valueToCheck == 598:   #half note
        return 'Half note tied to an eighth note triplet' #Fraction(1,2)
    
    elif valueToCheck == 640:   #half note
        return 'Half note tied to a eighth note' #Fraction(1,2)
    
    elif valueToCheck == 682:   #half note
        return 'Half note tied to a quarter note triplet' #Fraction(1,2)
    
    elif valueToCheck == 768:   #dotted half note
        return 'Dotted half note' #Fraction(3,4)
    
    elif valueToCheck == 896:   #dotted half note
        return 'Half note tied to a dotted quarter note' #Fraction(3,4)
    
    elif valueToCheck == 1024:   #whole note
        return 'Whole note' #Fraction(1,1)
    
    elif valueToCheck == 1365:   #whole note
        return 'Whole note tied to a half note triplet' #Fraction(1,1)
    
    elif valueToCheck == 1622:   #whole note
        return 'Dotted whole note tied to an eighth note triplet' #Fraction(1,1)
    
    elif valueToCheck == 1920:   #whole note
        return 'Dotted whole note tied to a dotted quarter note' #Fraction(1,1)
    
    else:
        return None


def cleanAccidental(data_row):
    accidentalString = str(data_row['accidental'])
    test = re.sub('<[^>]*>', '', accidentalString)
    return str(test) 


def cleanOctave(data_row):
    accidentalString = str(data_row['Octave'])
    test = re.sub('<[^>]*>', '', accidentalString)
    return str(test) 

def cleanRest(data_row):
    restVal = -1
    restAsString = str(data_row['Rest or note flag'])
    if restAsString != 'None':
        restVal ='rest'
    else:
        restVal = 'note'

    return str(restVal)

def cleanPart(data_row):
    accidentalString = str(data_row['instrument'])
    test = accidentalString[3:5]
    return str(test) 

def convertDurationAsPerQuarterNoteIsOneBeat(data_row):
    asFloat = float(data_row['duration'])
    #print asFloat
    asFloat = asFloat / 256
    asFloat = floor(asFloat * 100) / 100.0
    
    return round(asFloat, 2)



def midiNumberAssign(data_row):
    offsetForC = 0
    offsetForD = 2
    offsetForE = 4
    offsetForF = 5
    offsetForG = 7
    offsetForA = 9
    offsetForB = 11

    def adjustForAccidental(accidentalVal):
        if accidentalVal == 'None':
            return 0
        else:
            return int(accidentalVal)

    if data_row['Octave'] != "None":
        baseVal = data_row['Octave'] * 12
        if data_row['Note name'] == 'C':
            return baseVal + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'D':
            return baseVal + offsetForD + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'E':
            return baseVal + offsetForE + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'F':
            return baseVal + offsetForF + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'G':
            return baseVal + offsetForG + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'A':
            return baseVal + offsetForA + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'B':
            return baseVal + offsetForB + adjustForAccidental(data_row['accidental'])



def adjustForAccidental(accidentalVal):
    if accidentalVal == 'None':
        return 0
    else:
        return int(accidentalVal)

    if data_row['Octave'] != "None":
        baseVal = int(data_row['Octave']) * 12
        if data_row['noteName'] == 'C':
            return baseVal + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'D':
            return baseVal + offsetForD + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'E':
            return baseVal + offsetForE + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'F':
            return baseVal + offsetForF + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'G':
            return baseVal + offsetForG + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'A':
            return baseVal + offsetForA + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'B':
            return baseVal + offsetForB + adjustForAccidental(data_row['accidental'])




def getChordRootAsInt(data_row):
    x = data_row['Chord name']
    if x != -1:    
        chordLetterVal = data_row['Chord name'][0:2]
        #print chordLetterVal
        if chordLetterVal.strip() == 'C':
            return 0
        elif chordLetterVal.strip() =='C#':
            return 1
        elif chordLetterVal.strip() == 'Db':
            return 1
        elif chordLetterVal.strip() == 'D':
            return 2
        elif chordLetterVal.strip() == 'D#':
            return 3
        elif chordLetterVal.strip() == 'Eb':
            return 3
        elif chordLetterVal.strip() == 'E':
            return 4
        elif chordLetterVal.strip() == 'F':
            return 5
        elif chordLetterVal.strip() == 'F#':
            return 6
        elif chordLetterVal.strip() == 'Gb':
            return 6
        elif chordLetterVal.strip() == 'G':
            return 7
        elif chordLetterVal.strip() == 'G#':
            return 8
        elif chordLetterVal.strip() == 'Ab':
            return 8
        elif chordLetterVal.strip() == 'A':
            return 9
        elif chordLetterVal.strip() == 'A#':
            return 10
        elif chordLetterVal.strip() == 'Bb':
            return 10
        elif chordLetterVal.strip() == 'B':
            return 11
        else:
            return -1
       


    
def getChordTypeAsInt(data_row):
    x = data_row['Chord name']
    #print x
    if x != -1:
        chordType = re.findall('\s(.*)', x)
        try:
            chordType = str(chordType[0])
        except:
            chordType = 'fix'
        
        if chordType == 'maj7':
            return 0
        elif chordType == 'min7':
            return 1
        elif chordType == 'dom7':
            return 2
        elif chordType == 'dim7':
            return 3
        elif chordType == 'maj7b5':
            return 4
        elif chordType == 'maj7#5':
            return 5
        elif chordType == 'maj7sus4':
            return 6
        elif chordType == 'min7b5':
            return 7
        elif chordType == 'dom7b5':
            return 8
        elif chordType == 'dom7#5':
            return 9
        elif chordType == 'dom7sus4':
            return 10
        
        return -1


def chordTypeForDisplay(data_row):
    x = data_row['Chord name']
    if x != -1:
        chordType = re.findall('\s(.*)', x)   
        try:
            chordType = str(chordType[0])
        except:
            chordType = 'fix'
        return chordType
    
    
def chordBassAsInt(data_row):
    x = data_row['Chord name']
    if x != -1:
        x = str(x)
        indexOfBackslash = x.find('/')
        if indexOfBackslash == -1:
            return -1
        else:
            #function to deal with bass note
            return 'placeholder'
    

def concatenateNoteNamesAndAccidentals(data_row):
    retString = str(data_row['Note name'])
    octave = ''
    if data_row['accidental'] == '-1':
        retString = retString + 'b'
    elif data_row['accidental'] == '1':
        retString = retString + '#'
    if not np.isnan(data_row['Octave']):
        octave = int(data_row['Octave'])
        
    retString = retString + str(octave)
    return retString


def concatenateNoteNamesAndAccidentalsNoOctave(data_row):
    retString = str(data_row['Note name'])
    if data_row['accidental'] == '-1':
        retString = retString + 'b'
    elif data_row['accidental'] == '1':
        retString = retString + '#'

    return retString

def getFloorValueForMeasure(data_row):
    offsetForMeasure = 1
    floorValue = data_row['Location of note or rest']
    floorValue = (floorValue / 4) + offsetForMeasure
    return math.floor(floorValue)

def fixAccidentalEquivalencies(data_row):
    currentNoteName = data_row['Name of note without octave']
    if not np.isnan(data_row['Octave']):
        octave = int(data_row['Octave'])
    if currentNoteName == 'None':
        return 'None'
    if currentNoteName == 'C#':
        currentNoteName = 'Db'
    if currentNoteName == 'D#':
        currentNoteName = 'Eb'
    if currentNoteName == 'F#':
        currentNoteName = 'Gb'
    if currentNoteName == 'G#':
        currentNoteName = 'Ab'
    if currentNoteName == 'A#':
        currentNoteName = 'Bb'
    if currentNoteName == 'Cb':
        currentNoteName = 'B'
    if currentNoteName == 'Fb':
        currentNoteName = 'E'
    return currentNoteName + str(octave)

def fixAccidentalEquivalenciesNoOctave(data_row):
    currentNoteName = data_row['Name of note without octave']
    #currentOctave = data_row['octave']
    if currentNoteName == 'None':
        return 'None'
    if currentNoteName == 'C#':
        currentNoteName = 'Db'
    if currentNoteName == 'D#':
        currentNoteName = 'Eb'
    if currentNoteName == 'F#':
        currentNoteName = 'Gb'
    if currentNoteName == 'G#':
        currentNoteName = 'Ab'
    if currentNoteName == 'A#':
        currentNoteName = 'Bb'
    if currentNoteName == 'Cb':
        currentNoteName = 'B'
    if currentNoteName == 'Fb':
        currentNoteName = 'E'
    return currentNoteName


def calculateDistanceOfNoteToChordRoot(data_row):
    offsetForPositiveValue = 12
    midiNumber = data_row['Midi number']
    chordRoot = data_row['Chord root as minimized midi number']
    midiNumber = (midiNumber % 12) + 12
    distanceToRoot = midiNumber - chordRoot
    distanceToRoot = distanceToRoot % 12
    return distanceToRoot
    

def calculateDistanceOfNoteToChordBass(data_row):
    pass

def createReadableChordDegreeNames(data_row):
    noteRootDistance = data_row['Semitone distance between note and chord root']
    if noteRootDistance == 0:
        return 'root'
    if noteRootDistance == 1:
        return 'flat ninth'
    if noteRootDistance == 2:
        return 'ninth'
    if noteRootDistance == 3:
        return 'sharp ninth or minor third'
    if noteRootDistance == 4:
        return 'major third'
    if noteRootDistance == 5:
        return 'eleventh'
    if noteRootDistance == 6:
        return 'sharp eleventh or flat fifth'
    if noteRootDistance == 7:
        return 'fifth'
    if noteRootDistance == 8:
        return 'flat thirteenth or sharp fifth'
    if noteRootDistance == 9:
        return 'thirteenth'
    if noteRootDistance == 10:
        return 'seventh'
    if noteRootDistance == 11:
        return 'major seventh'
    return noteRootDistance

def getNextTenChordMidiNumbers(chordRootAsIntValues, data_row):
    currentIndex = data_row.name
    currentChordRootAsInt = data_row['Chord root as minimized midi number']
    listOfAllChords = chordRootAsIntValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]

    return sliceOfChords  


def createNextTenChordRootsAsNotes(data_row):
    stringToReturn = ''
    midiNumberSequence = data_row['Next ten chord root notes as midi numbers']
    for items in midiNumberSequence:
        if items == 0.0:
            stringToReturn = stringToReturn + 'Cx'
        elif items == 1.0:
            stringToReturn = stringToReturn + 'Db'
        elif items == 2.0:
            stringToReturn = stringToReturn + 'Dx'
        elif items == 3.0:
            stringToReturn = stringToReturn + 'Eb'
        elif items == 4.0:
            stringToReturn = stringToReturn + 'Ex'
        elif items == 5.0:
            stringToReturn = stringToReturn + 'Fx'
        elif items == 6.0:
            stringToReturn = stringToReturn + 'Gb'
        elif items == 7.0:
            stringToReturn = stringToReturn + 'Gx'
        elif items == 8.0:
            stringToReturn = stringToReturn + 'Ab'
        elif items == 9.0:
            stringToReturn = stringToReturn + 'Ax'
        elif items == 10.0:
            stringToReturn = stringToReturn + 'Bb'
        elif items == 11.0:
            stringToReturn = stringToReturn + 'Bx'
    return stringToReturn

def createNextTenChordRootsAsDegrees(data_row):
    stringToReturn = ''
    midiNumberSequence = data_row['Next ten chord root notes as midi numbers']
    firstValue = midiNumberSequence[0]
    for eachItem in midiNumberSequence:
        difference = int(eachItem) - int(firstValue)
        if difference < 0:
            difference = difference + 12
        if difference < 10:
            difference = '0' + str(difference)
        else: 
            difference = str(difference)
        stringToReturn = stringToReturn + str(difference)
    return stringToReturn


def getNextTenChordTypes(chordTypeValues, data_row):
    currentIndex = data_row.name
    currentChordType = data_row['Chord type name']
    listOfAllChords = chordTypeValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]
    sliceOfChordsAsString = ''
    for eachValue in sliceOfChords:
        sliceOfChordsAsString = sliceOfChordsAsString + eachValue
    
    #return sliceOfChords  
    return sliceOfChordsAsString

def getNextTenChordTypesV2(chordTypeValues, data_row):
    currentIndex = data_row.name
    currentChordType = data_row['Chord type name']

    listOfAllChords = chordTypeValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]
    sliceOfChordsAsString = ''
    for eachValue in sliceOfChords:
        paddedValue= eachValue.ljust(8, 'x')
        sliceOfChordsAsString = sliceOfChordsAsString + paddedValue
    
    #return sliceOfChords  
    return sliceOfChordsAsString


def getNextFiftyMidiNumbers(midiNumbersAsValues, data_row):
    currentIndex = data_row.name
    listOfAllMidiNumbers = midiNumbersAsValues
    sliceOfMidiNumbers = listOfAllMidiNumbers[currentIndex:]
    sliceOfMidiNumbers = sliceOfMidiNumbers.tolist()
    sliceOfMidiNumbers = sliceOfMidiNumbers[0:50]
    #print sliceOfMidiNumbers
    sliceOfMidiNumbersAsString = ''
    for eachValue in sliceOfMidiNumbers:
        if not (math.isnan(eachValue)):
            sliceOfMidiNumbersAsString = sliceOfMidiNumbersAsString + str(int(eachValue))
        else: 
            sliceOfMidiNumbersAsString = sliceOfMidiNumbersAsString + 'xx'
    

    return sliceOfMidiNumbersAsString




def getNextFiftyDurations(durationsAsValues, data_row):
    currentIndex = data_row.name
    listOfAllDurations = durationsAsValues
    sliceOfDurations = listOfAllDurations[currentIndex:]
    sliceOfDurations = sliceOfDurations.tolist()
    sliceOfDurations = sliceOfDurations[0:10]
    
    sliceOfDurationsAsString = ''
    for eachValue in sliceOfDurations:
        if not (math.isnan(eachValue)):
            paddedValue = str(eachValue)
            paddedValue = paddedValue.ljust(9, 'x')
            sliceOfDurationsAsString = sliceOfDurationsAsString + paddedValue
    
    return sliceOfDurationsAsString



def getNextFiftyLocations(locationsAsValues, data_row):
    currentIndex = data_row.name
    amountToNegate = data_row['Beginning of location of note or rest']
    listOfAllLocations = locationsAsValues
    sliceOfLocations = listOfAllLocations[currentIndex:]
    sliceOfLocations = sliceOfLocations.tolist()
    sliceOfLocations = sliceOfLocations[0:50]
    sliceOfLocationsAsString = ''
    for eachValue in sliceOfLocations:
        if not (math.isnan(eachValue)):
            offsetValue = eachValue - amountToNegate
            #roundedLocation = round(offsetValue, 4)
            paddedValue = str(offsetValue)
            paddedValue = paddedValue.ljust(9, 'x')
            sliceOfLocationsAsString = sliceOfLocationsAsString + paddedValue

    return sliceOfLocationsAsString.replace('False','0xxxx')


def getNextFiftyNotesAsDistancesFromStartingNote(midiNumbersAsValues, data_row):
    currentIndex = data_row.name
    listOfAllMidiNumbers = midiNumbersAsValues
    sliceOfMidiNumbers = listOfAllMidiNumbers[currentIndex:]
    sliceOfMidiNumbers = sliceOfMidiNumbers.tolist()
    sliceOfMidiNumbersWithoutNan = []
    
    for eachValue in sliceOfMidiNumbers:
        if not (math.isnan(eachValue)):
            sliceOfMidiNumbersWithoutNan.append(eachValue)

    stringToReturn = ''
    
    sliceOfMidiNumbersWithoutNan = sliceOfMidiNumbersWithoutNan[0:50]
    
    for i in range(0, len(sliceOfMidiNumbersWithoutNan)):
        currentValue = sliceOfMidiNumbersWithoutNan[i]
        try:
            nextValue = sliceOfMidiNumbersWithoutNan[i+1]
            difference = int(nextValue) - int(currentValue)
            differenceAsString = str(difference)
            if len(differenceAsString) == 1:
                differenceAsString = '+' + differenceAsString
            if len(differenceAsString) == 2:
                differenceAsString = differenceAsString + 'x'
            stringToReturn = stringToReturn + str(differenceAsString)
        except:
            pass

    return stringToReturn




def getCumulativeDuration(fullListOfDurationsAsFractions, data_row):
    currentIndex = data_row.name
    sumSoFar = fullListOfDurationsAsFractions[0:currentIndex].sum()
    #print 'At index', currentIndex, ' the sum is ' , sumSoFar

    return sumSoFar


def convertGlobalLocationToFloat( globalLocation, data_row):
    globalLocation = globalLocation
    locationInCurrentSong = data_row['Location of note or restV2']
    locationInCurrentSong = float(locationInCurrentSong)
    
    location = globalLocation + locationInCurrentSong
    return round(location, 8)


def getDurationAsFloat(data_row):
    x = data_row['Duration as a fraction of a whole note']
    return float(x)

def getLocationAsFloat(data_row):
    x = data_row['Beginning of location of note or rest']
    return float(x)


def fixTiedNotes(tiedValues, data_row):
    currentTieFlag = data_row['tieFlag']
    if currentTieFlag == '<tie type="stop"></tie>' or currentTieFlag == '<tie type="start"></tie>':
        return 'merge'
    else:
        return None
    
def fixRestNotes(tiedValues, data_row):
#     currentRestFlag = data_row['Rest or note flag']
#     currentIndex = data_row.name
#     if currentRestFlag == 'rest':
#         return tiedValues
#     else:
    return None


def converToString(data_row):
    listOfVals = data_row['Next ten chord root notes as midi numbers']
    returnString = ''
    for value in listOfVals:
        if value < 10:
            returnString = returnString + 'x' + str(value)
        else:
            returnString = returnString + str(value)
            
    return returnString
        


In [4]:
class CreateDataSetFromXML:
    def __init__(self, xmlFile="", performerName="", composerName="", 
                 genre="", numberOfParts=-1, title="", yearOfComposition="", 
                 yearOfPerformance="", ongoingTempo=-1, 
                 numberOfQuarterBeatsInBar = -1, tempoChangeIsOccuring = False, 
                 globalLocation = -1):

        self.xmlFile = xmlFile
        self.genre = genre
        self.numberOfParts = numberOfParts
        self.parsedXMLData = ""
        self.df = ""
        self.performerName = performerName
        self.composerName = composerName
        self.genre = genre
        self.numberOfParts = numberOfParts
        self.title = title
        self.yearOfComposition = yearOfComposition
        self.yearOfPerformance = yearOfPerformance
        self.ongoingTempo = ongoingTempo
        self.numberOfQuarterBeatsInBar = numberOfQuarterBeatsInBar
        self.tempoChangeIsOccuring = tempoChangeIsOccuring
        self.globalLocation = globalLocation
    

    def convertXMLToDataFrame(self):
        parsedXMLData = BeautifulSoup(open(self.xmlFile))
        allNotes = parsedXMLData.findAll('measure')
        listOfNoteEvents = []
        listOfSoloNotes = []
        listOfSoloNotesWithMergedRests = []
        for measure in allNotes:
            #print measure["number"]

            notes = measure.findAll('note')
            words = measure.findAll('words')
            
            for note in notes:
                measureDictionary = {}
                if note.pitch:
                    measureDictionary["Chord name"] = None
                    measureDictionary["instrument"] = measure.instrument["id"]
                    measureDictionary["Measure"] = int(measure["number"])
                    measureDictionary["Note name"] = note.step.text
                    measureDictionary["Octave"] = int(note.octave.text)
                    if note.duration != None:
                        measureDictionary['duration'] = float(note.duration.text)
                    measureDictionary['accidental'] = note.alter
                    measureDictionary["Rest or note flag"] = 'note'
                    measureDictionary['tieFlag'] = str(note.tie)
                else:
                    measureDictionary["Chord name"] = None
                    measureDictionary["instrument"] = measure.instrument["id"]
                    measureDictionary["Measure"] = int(measure["number"])
                    measureDictionary["Note name"] = None
                    measureDictionary["Octave"] = None
                    measureDictionary['duration'] = float(note.duration.text)
                    measureDictionary['accidental'] = None
                    measureDictionary["Rest or note flag"] = 'rest'
                    measureDictionary['tieFlag'] = None
                if measureDictionary['instrument'] == 'P1-I1':
                    measureDictionary['Instrument name'] = "Piano Solo Jazz (Treble)"
                    listOfNoteEvents.append(measureDictionary)
                    #listOfSoloNotes.append(measureDictionary)

            #print measure["number"]
            for word in range(0, len(words)):
                chordDictionary = {}
                chordDictionary['Chord name'] = words[word].text
                #print words[word].text
                chordDictionary["instrument"] = measure.instrument["id"]
                chordDictionary["Instrument name"] = "Jazz Harmony Bass Note" 
                chordDictionary["Measure"] = int(measure["number"])
                chordDictionary["Note name"] = words[word].parent.parent.parent.note.step.text
                #print chordDictionary["Note name"]
                chordDictionary["Octave"] = int(words[word].parent.parent.parent.note.octave.text)
                chordDictionary['duration'] = float(words[word].parent.parent.parent.note.duration.text)
                chordDictionary['accidental'] = words[word].parent.parent.parent.note.alter
                chordDictionary["Rest or note flag"] = 'note'
                chordDictionary['tieFlag'] = str(words[word].parent.parent.parent.note.tie)
                
                listOfNoteEvents.append(chordDictionary)
        
        self.df = pd.DataFrame(listOfNoteEvents)



        
        
    def fixLocationAndDuration(self):
        self.df['Duration as a fraction of a whole note'] = self.df.apply(getDurationsForMongoQueries, axis=1)
        
        soloPart = self.df[self.df['Instrument name'] == 'Piano Solo Jazz (Treble)']
        soloPart = soloPart.reset_index(drop=True)
        harmonyPart = self.df[self.df['Instrument name'] == 'Jazz Harmony Bass Note']
        harmonyPart = harmonyPart.reset_index(drop=True)
        
        #soloPart['valueToAddToTiedNote']=soloPart.apply(fixTiedNotes, axis=1)
        
        soloTiedValues = soloPart['tieFlag'].values
        restOrNoteValues = soloPart['Rest or note flag'].values
        
        soloPart['merge due to tie'] = soloPart.apply(partial(fixTiedNotes,soloTiedValues), axis=1)
        soloPart['merge due to rest'] = soloPart.apply(partial(fixRestNotes,soloTiedValues), axis=1)
        

        
        
        
        
        #soloPart = soloPart[soloPart.tieFlag != '<tie type="stop"></tie>']        
        #soloPart['Duration as a fraction of a whole note'] = soloPart['Duration as a fraction of a whole note']+ soloPart['moveTieForFix']
        
        
        soloPart['Location of note or rest'] = soloPart['Duration as a fraction of a whole note'].cumsum()
        harmonyPart['Location of note or rest'] = harmonyPart['Duration as a fraction of a whole note'].cumsum()
        soloPart['Location of note or rest'] = soloPart['Location of note or rest'] + 1
        harmonyPart['Location of note or rest'] = harmonyPart['Location of note or rest'] + 1
        soloPart['Beginning of location of note or rest'] = soloPart['Location of note or rest'] - soloPart['Duration as a fraction of a whole note']
        harmonyPart['Beginning of location of note or rest'] = harmonyPart['Location of note or rest'] - harmonyPart['Duration as a fraction of a whole note']
        
        self.df = pd.concat([soloPart, harmonyPart], axis=0)
        self.df = self.df.sort(['Beginning of location of note or rest', 'Instrument name'], ascending=[1,1])
        
        self.df['Location as float value'] = self.df.apply(getLocationAsFloat, axis=1)
        
        self.df = self.df.reset_index(drop=True)
        self.df['Name of duration'] = self.df.apply(getDurationNamesForMongoQueries, axis=1)
        
        self.df['Duration as float value'] = self.df.apply(getDurationAsFloat, axis=1)

 
        #display(HTML(dataFrameWithMultipleRestsFixed.to_html()))
        
    def addMetaDataAndClean(self):
        self.df['Performer name'] = self.performerName
        self.df['Composer name'] = self.composerName
        self.df['Genre'] = self.genre
        self.df['Title'] = self.title
        self.df['Year of composition'] = self.yearOfComposition
        self.df['Year of performance'] = self.yearOfPerformance
        self.df['Tempo at this location'] = self.ongoingTempo
        self.df['Title'] = self.title
        self.df['accidental'] = self.df.apply(cleanAccidental, axis=1) 
        self.df['Midi number'] = self.df.apply(midiNumberAssign, axis=1)
        self.df['Name of note'] = self.df.apply(concatenateNoteNamesAndAccidentals, axis=1)
        self.df['Name of note without octave'] = self.df.apply(concatenateNoteNamesAndAccidentalsNoOctave, axis=1)
        self.df['convertAllSharpsToFlats'] = self.df.apply(fixAccidentalEquivalencies, axis=1)
        self.df['convertAllSharpsToFlatsNoOctave'] = self.df.apply(fixAccidentalEquivalenciesNoOctave, axis=1)
        self.df['Name of note'] = self.df['convertAllSharpsToFlats']
        self.df['Name of note without octave'] = self.df['convertAllSharpsToFlatsNoOctave']
        self.df['Text written on score'] = None
        self.df['Link to audio file'] = None
        self.df['Time position in audio file where note appears'] = None
        self.df['Global location'] = None
        self.df['Structural note flag'] = None
        self.df['Auxillary note flag'] = None
        self.df['User defined search tags'] = None
        
        del self.df["Note name"]
        del self.df["accidental"]
        del self.df["duration"]
        del self.df["instrument"]
        del self.df["convertAllSharpsToFlatsNoOctave"]
        del self.df["convertAllSharpsToFlats"]
        del self.df['Location of note or rest']

        
    def sortByInstrumentsAndCalculateMidiDistancesBetweenNotes(self):
        
        soloPart = self.df[self.df['Instrument name'] == 'Piano Solo Jazz (Treble)']
        soloPart = soloPart.reset_index(drop=True)
        harmonyPart = self.df[self.df['Instrument name'] == 'Jazz Harmony Bass Note']
        harmonyPart = harmonyPart.reset_index(drop=True)
        
        soloPart['Semitone distance between current and next note'] = soloPart['Midi number'].shift(-1)
        soloPart['Semitone distance between current and previous note'] = soloPart['Midi number'].shift(1)
        soloPart['Semitone distance between current and next note'] = soloPart['Semitone distance between current and next note'] - soloPart['Midi number']
        soloPart['Semitone distance between current and previous note'] = soloPart['Semitone distance between current and previous note'] - soloPart['Midi number']
        
        harmonyPart['Semitone distance between current and next note'] = harmonyPart['Midi number'].shift(-1)
        harmonyPart['Semitone distance between current and previous note'] = harmonyPart['Midi number'].shift(1)
        harmonyPart['Semitone distance between current and next note'] = harmonyPart['Semitone distance between current and next note'] - harmonyPart['Midi number']
        harmonyPart['Semitone distance between current and previous note'] = harmonyPart['Semitone distance between current and previous note'] - harmonyPart['Midi number']
        
        self.df = pd.concat([soloPart, harmonyPart], axis=0)
        self.df = self.df.sort(['Beginning of location of note or rest', 'Instrument name'], ascending=[1,1])
        self.df = self.df.reset_index(drop=True)
        
        
    
    def updateChordData(self):
        self.df['Chord name'].fillna(method='ffill', inplace=True)
        self.df['Chord root as minimized midi number'] = self.df.apply(getChordRootAsInt, axis=1)
        self.df['Chord bass as minimized midi number'] = self.df.apply(chordBassAsInt, axis=1)
        self.df['Chord type name'] = self.df.apply(chordTypeForDisplay, axis=1)
        self.df['Semitone distance between note and chord root'] = self.df.apply(calculateDistanceOfNoteToChordRoot, axis=1)
        #self.df['Semitone distance between note and chord bass'] = self.df.apply(calculateDistanceOfNoteToChordBass, axis=1)  
        self.df['Harmonic function of note given the underlying chord'] = self.df.apply(createReadableChordDegreeNames, axis=1)

        chordRootAsIntValues = self.df['Chord root as minimized midi number'].values
        chordNamesAsValues = self.df['Chord name'].values
        chordTypesAsValues = self.df['Chord type name'].values
        #self.df['Next ten chord names'] = self.df.apply(partial(getNextTenChordTypesV2,chordNamesAsValues), axis=1)
        self.df['Next ten chord types'] = self.df.apply(partial(getNextTenChordTypesV2,chordTypesAsValues), axis=1)
        self.df['Next ten chord root notes as midi numbers'] = self.df.apply(partial(getNextTenChordMidiNumbers,chordRootAsIntValues), axis=1)
        self.df['Next ten chord roots as notes'] = self.df.apply(createNextTenChordRootsAsNotes, axis=1)
        self.df['Next ten chord roots as distances'] = self.df.apply(createNextTenChordRootsAsDegrees, axis=1)
        self.df['Next ten chord root notes as midi numbers 2'] = self.df.apply(converToString, axis=1)
        self.df['Next ten chord root notes as midi numbers'] = self.df['Next ten chord root notes as midi numbers 2']
        del self.df['Next ten chord root notes as midi numbers 2']
        

    def createStringsNeededForMongoRegexQueries(self):
        
        soloPart = self.df[self.df['Instrument name'] == 'Piano Solo Jazz (Treble)']
        soloPart = soloPart.reset_index(drop=True)
        harmonyPart = self.df[self.df['Instrument name'] == 'Jazz Harmony Bass Note']
        harmonyPart = harmonyPart.reset_index(drop=True)
     
        allMidiNumbersInSoloPart = soloPart['Midi number'].values
        allMidiNumbersInHarmonyPart = harmonyPart['Midi number'].values
        
        allDurationsInSoloPart = soloPart['Duration as a fraction of a whole note'].values
        allDurationsInHarmonyPart = harmonyPart['Duration as a fraction of a whole note'].values
        
        allLocationsOfRestsAndNotesInSoloPart = soloPart['Beginning of location of note or rest'].values
        allLocationsOfRestsAndNotesInHarmonyPart = harmonyPart['Beginning of location of note or rest'].values
        

        soloPart['Next fifty midi numbers'] = soloPart.apply(partial(getNextFiftyMidiNumbers,allMidiNumbersInSoloPart), axis=1)
        harmonyPart['Next fifty midi numbers'] = harmonyPart.apply(partial(getNextFiftyMidiNumbers,allMidiNumbersInHarmonyPart), axis=1)
        
        soloPart['Next fifty durations'] = soloPart.apply(partial(getNextFiftyDurations, allDurationsInSoloPart), axis=1)
        harmonyPart['Next fifty durations'] = harmonyPart.apply(partial(getNextFiftyDurations, allDurationsInHarmonyPart), axis=1)
                
        soloPart['Next fifty music event locations'] = soloPart.apply(partial(getNextFiftyLocations, allLocationsOfRestsAndNotesInSoloPart), axis=1)
        harmonyPart['Next fifty music event locations'] = harmonyPart.apply(partial(getNextFiftyLocations, allLocationsOfRestsAndNotesInHarmonyPart), axis=1) 
        
        
        soloPart['Next fifty midi distances'] = soloPart.apply(partial(getNextFiftyNotesAsDistancesFromStartingNote,allMidiNumbersInSoloPart), axis=1)
        harmonyPart['Next fifty midi distances'] = harmonyPart.apply(partial(getNextFiftyNotesAsDistancesFromStartingNote, allMidiNumbersInHarmonyPart), axis=1) 
       

        self.df = pd.concat([soloPart, harmonyPart], axis=0)
        self.df = self.df.sort(['Beginning of location of note or rest', 'Instrument name'], ascending=[1,1])
        self.df = self.df.reset_index(drop=True)
        
        
        self.df['Global location'] = self.globalLocation + self.df['Location as float value']
        self.df['Global event location'] = self.globalLocation + self.df.index
        

    def returnPreparedDataFrame(self):
        return self.df

In [5]:
stellaData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettStellaByStarlight.xml", 
                                  performerName="Keith Jarrett", composerName="Victor Young", genre="Jazz", 
                                  numberOfParts=2, title="Stella By Starlight", 
                                  yearOfComposition=1944, yearOfPerformance=1983, 
                                  ongoingTempo=151,numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 20000)
stellaData.convertXMLToDataFrame()
stellaData.fixLocationAndDuration()
stellaData.addMetaDataAndClean()
stellaData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
stellaData.updateChordData()
stellaData.createStringsNeededForMongoRegexQueries()
stellaDf = stellaData.returnPreparedDataFrame()

In [6]:
#stellaDf
#stellaDf[40:90][['Chord name' ,'duration as float', 'duration', 'location as float', 'Beginning of location of note or rest', 'Duration as a fraction of a whole note','Instrument name', 'Location of note or rest','Measure','Name of note' ]]
#check = stellaDf[stellaDf['Instrument name'] == 'Piano Solo Jazz (Treble)']
#check.groupby('Measure').sum()['Duration as float value'].cumsum()

In [7]:
#stellaDf

In [8]:
groovinData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettGroovinHigh.xml", 
                                  performerName="Keith Jarrett", composerName="Dizzy Gillespie", genre="Jazz", 
                                  numberOfParts=2, title="Groovin High", 
                                  yearOfComposition=1945, yearOfPerformance=1999, 
                                  ongoingTempo=282, 
                                  numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 30000)
groovinData.convertXMLToDataFrame()
groovinData.fixLocationAndDuration()
groovinData.addMetaDataAndClean()
groovinData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
groovinData.updateChordData()
groovinData.createStringsNeededForMongoRegexQueries()
groovinDf = groovinData.returnPreparedDataFrame()

In [9]:
#stellaDf[40:90][['Chord name' ,'duration as float', 'duration', 'location as float', 'Beginning of location of note or rest', 'Duration as a fraction of a whole note','Instrument name', 'Location of note or rest','Measure','Name of note' ]]
#check = groovinDf[groovinDf['Instrument name'] == 'Piano Solo Jazz (Treble)']
#check.groupby('Measure').sum()['Duration as float value'].cumsum()

In [10]:
autumnData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAutumnLeaves.xml", 
                                  performerName="Keith Jarrett", composerName="Joseph Kosma", genre="Jazz", 
                                  numberOfParts=2, title="Autumn Leaves", 
                                  yearOfComposition=1945, yearOfPerformance=1996, 
                                  ongoingTempo=214, 
                                  numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 40000)
autumnData.convertXMLToDataFrame()
autumnData.fixLocationAndDuration()
autumnData.addMetaDataAndClean()
autumnData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
autumnData.updateChordData()
autumnData.createStringsNeededForMongoRegexQueries()
autumnDf = autumnData.returnPreparedDataFrame()

In [11]:
#stellaDf[40:90][['Chord name' ,'duration as float', 'duration', 'location as float', 'Beginning of location of note or rest', 'Duration as a fraction of a whole note','Instrument name', 'Location of note or rest','Measure','Name of note' ]]
#check = autumnDf[autumnDf['Instrument name'] == 'Piano Solo Jazz (Treble)']
#check.groupby('Measure').sum()['Duration as float value'].cumsum()

In [12]:
#autumnDf

In [13]:
ifIWereData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettIfIWereABell.xml", 
                                  performerName="Keith Jarrett", composerName="Frank Loesser", genre="Jazz", 
                                  numberOfParts=2, title="If I Were A Bell", 
                                  yearOfComposition=1950, yearOfPerformance=2002, 
                                  ongoingTempo=154, numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 50000)
ifIWereData.convertXMLToDataFrame()
ifIWereData.fixLocationAndDuration()
ifIWereData.addMetaDataAndClean()
ifIWereData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
ifIWereData.updateChordData()
ifIWereData.createStringsNeededForMongoRegexQueries()
ifIWereDf = ifIWereData.returnPreparedDataFrame()

In [14]:
#check = ifIWereDf[ifIWereDf['Instrument name'] == 'Piano Solo Jazz (Treble)']
#check.groupby('Measure').sum()['Duration as float value'].cumsum()

In [15]:
allTheThingsData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAllTheThingsYouAre.xml", 
                                  performerName="Keith Jarrett", composerName="Jerome Kern", genre="Jazz", 
                                  numberOfParts=2, title="All The Things You Are", 
                                  yearOfComposition=1939, yearOfPerformance=1983, 
                                  ongoingTempo=120, tempoChangeIsOccuring = False, 
                                  numberOfQuarterBeatsInBar=4, globalLocation = 60000)
allTheThingsData.convertXMLToDataFrame()
allTheThingsData.fixLocationAndDuration()
allTheThingsData.addMetaDataAndClean()
allTheThingsData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
allTheThingsData.updateChordData()
allTheThingsData.createStringsNeededForMongoRegexQueries()
allTheThingsDf = allTheThingsData.returnPreparedDataFrame()

In [16]:
#check = allTheThingsDf[allTheThingsDf['Instrument name'] == 'Piano Solo Jazz (Treble)']
#check.groupby('Measure').sum()['Duration as float value'].cumsum()

In [17]:
somedayMyPrinceData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettSomedayMyPrinceWillCome.xml", 
                                  performerName="Keith Jarrett", composerName="Frank Churchill", genre="Jazz", 
                                  numberOfParts=2, title="Someday My Prince Will Come", 
                                  yearOfComposition=1937, yearOfPerformance=2002, 
                                  ongoingTempo=146, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=3, globalLocation = 70000)
somedayMyPrinceData.convertXMLToDataFrame()
somedayMyPrinceData.fixLocationAndDuration()
somedayMyPrinceData.addMetaDataAndClean()
somedayMyPrinceData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
somedayMyPrinceData.updateChordData()
somedayMyPrinceData.createStringsNeededForMongoRegexQueries()
somedayMyPrinceDf = somedayMyPrinceData.returnPreparedDataFrame()

In [18]:
#check = somedayMyPrinceDf[somedayMyPrinceDf['Instrument name'] == 'Piano Solo Jazz (Treble)']
#check.groupby('Measure').sum()['Duration as float value'].cumsum()

In [19]:
daysOfWineData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettDaysOfWineAndRoses.xml", 
                                  performerName="Keith Jarrett", composerName="Henry Mancini", genre="Jazz", 
                                  numberOfParts=2, title="Days Of Wine And Roses", 
                                  yearOfComposition=1962, yearOfPerformance=1994, 
                                  ongoingTempo=168, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=4, globalLocation = 80000)
daysOfWineData.convertXMLToDataFrame()
daysOfWineData.fixLocationAndDuration()
daysOfWineData.addMetaDataAndClean()
daysOfWineData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
daysOfWineData.updateChordData()
daysOfWineData.createStringsNeededForMongoRegexQueries()
daysOfWineDf = daysOfWineData.returnPreparedDataFrame()

In [20]:
#check = daysOfWineDf[daysOfWineDf['Instrument name'] == 'Piano Solo Jazz (Treble)']
#check.groupby('Measure').sum()['Duration as float value'].cumsum()

In [21]:
df = pd.concat([stellaDf, groovinDf, autumnDf, ifIWereDf, allTheThingsDf, somedayMyPrinceDf, daysOfWineDf], axis=0)
#df = stellaDf
df.to_pickle('data_set_as_pk.pkl')

In [22]:
df['Title'].value_counts()

All The Things You Are         2341
If I Were A Bell               2308
Groovin High                   2127
Someday My Prince Will Come    2105
Stella By Starlight            1688
Days Of Wine And Roses         1626
Autumn Leaves                  1426
dtype: int64

In [23]:
#df[df['Title'] == 'If I Were A Bell'][['Duration as float value' , 'Location as float value', 'Chord name' ,'Beginning of location of note or rest', 'Duration as a fraction of a whole note','Instrument name', 'Measure','Name of note' ]][1580:]

In [24]:
#df[df['Title']=='If I Were A Bell'].groupby('Measure').sum()

In [25]:
#df['Duration as a fraction of a whole note'].value_counts()

In [26]:
#df.dtypes

In [27]:
df.head()

,Beginning of location of note or rest,Chord name,Duration as a fraction of a whole note,Instrument name,Measure,Octave,Rest or note flag,merge due to rest,merge due to tie,tieFlag,Location as float value,Name of duration,Duration as float value,Performer name,Composer name,Genre,Title,Year of composition,Year of performance,Tempo at this location,Midi number,Name of note,Name of note without octave,Text written on score,Link to audio file,Time position in audio file where note appears,Global location,Structural note flag,Auxillary note flag,User defined search tags,Semitone distance between current and next note,Semitone distance between current and previous note,Chord root as minimized midi number,Chord bass as minimized midi number,Chord type name,Semitone distance between note and chord root,Harmonic function of note given the underlying chord,Next ten chord types,Next ten chord root notes as midi numbers,Next ten chord roots as notes,Next ten chord roots as distances,Next fifty midi numbers,Next fifty durations,Next fifty music event locations,Next fifty midi distances,Global event location
0,1,E min7b5,1,Jazz Harmony Bass Note,1,2,note,NaN,NaN,None,1.000,Whole note,1.000,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,28,E2,E,None,None,None,20001.000,None,None,None,5,NaN,4,-1,min7b5,0,root,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,x4x9x0x510x3x810x4x9x2,ExAxCxFxBbEbAbBbExAxDx,0005080106110406000510,2833242929342732342828263434292828332631312424...,1xxxxxxxx1xxxxxxxx1xxxxxxxx1xxxxxxxx1xxxxxxxx1...,0xxxxxxxx1xxxxxxxx2xxxxxxxx3xxxxxxxx4xxxxxxxx5...,+5x-9x+5x+0x+5x-7x+5x+2x-6x+0x-2x+8x+0x-5x-1x+...,20000
1,1,E min7b5,1/8,Piano Solo Jazz (Treble),1,4,note,None,None,None,1.000,Eighth note,0.125,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,55,G4,G,None,None,None,20001.000,None,None,None,NaN,NaN,4,-1,min7b5,3,sharp ninth or minor third,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,x4x9x0x510x3x810x4x9x2,ExAxCxFxBbEbAbBbExAxDx,0005080106110406000510,55xxxx575857xx55585757xx55xx5153626057535150xx...,1/8xxxxxx1/8xxxxxx1/8xxxxxx1/8xxxxxx1/4xxxxxx1...,0xxxxxxxx1/8xxxxxx1/4xxxxxx3/8xxxxxx1/2xxxxxx3...,+2x+1x-1x-2x+3x-1x+0x-2x-4x+2x+9x-2x-3x-4x-2x-...,20001
2,9/8,E min7b5,1/8,Piano Solo Jazz (Treble),1,NaN,rest,None,None,None,1.125,Eighth note,0.125,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,NaN,None,None,None,None,None,20001.125,None,None,None,NaN,NaN,4,-1,min7b5,NaN,NaN,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,x4x9x0x510x3x810x4x9x2,ExAxCxFxBbEbAbBbExAxDx,0005080106110406000510,xxxx575857xx55585757xx55xx5153626057535150xxxx...,1/8xxxxxx1/8xxxxxx1/8xxxxxx1/4xxxxxx1/4xxxxxx1...,0xxxxxxxx1/8xxxxxx1/4xxxxxx3/8xxxxxx5/8xxxxxx7...,+1x-1x-2x+3x-1x+0x-2x-4x+2x+9x-2x-3x-4x-2x-1x+...,20002
3,5/4,E min7b5,1/8,Piano Solo Jazz (Treble),1,NaN,rest,None,None,None,1.250,Eighth note,0.125,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,NaN,None,None,None,None,None,20001.250,None,None,None,NaN,NaN,4,-1,min7b5,NaN,NaN,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,x4x9x0x510x3x810x4x9x2,ExAxCxFxBbEbAbBbExAxDx,0005080106110406000510,xx575857xx55585757xx55xx5153626057535150xxxx51...,1/8xxxxxx1/8xxxxxx1/4xxxxxx1/4xxxxxx1/8xxxxxx1...,0xxxxxxxx1/8xxxxxx1/4xxxxxx1/2xxxxxx3/4xxxxxx7...,+1x-1x-2x+3x-1x+0x-2x-4x+2x+9x-2x-3x-4x-2x-1x+...,20003
4,11/8,E min7b5,1/8,Piano Solo Jazz (Treble),1,4,note,None,None,None,1.375,Eighth note,0.125,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,57,A4,A,None,None,None,20001.375,None,None,None,1,NaN,4,-1,min7b5,5,eleventh,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,x4x9x0x510x3x810x4x9x2,ExAxCxFxBbEbAbBbExAxDx,0005080106110406000510,575857xx55585757xx55xx5153626057535150xxxx5152...,1/8xxxxxx1/4xxxxxx1/4xxxxxx1/8xxxxxx1/8xxxxxx1...,0xxxxxxxx1/8xxxxxx3/8xxxxxx5/8xxxxxx3/4xxxxxx7...,+1x-1x-2x+3x-1x+0x-2x-4x+2x+9x-2x-3x-4x-2x-1x+...,20004


In [28]:
df.to_json(orient='records', path_or_buf='FinalDataSet/JSONFilesLoadedIntoDataBase/fullcorpus.json')